In [3]:
!pip install mysql.connector
!pip install pandas

In [1]:
import mysql.connector
from mysql.connector import Error
import json
import pandas as pd


In [174]:
# Function to load configuration
def load_config():
    with open('config.json', 'r') as config_file:
        config = json.load(config_file)
    return config

config = load_config()


#Function to create MySQL connection
def create_connection():
    connection=None
    try: 
        connection = mysql.connector.connect(
            host = config["DB_HOST"],
            user = config["DB_USER"],
            password = config["DB_PASS"],
            database = config["DB_NAME"]
        )
        print("Connection to MySQL DB successful")
    except error as e:
        print("The error {e} occurred")
    return connection

#function to execute a query in MySQL
def select_execute_query(connection, query):
    cursor = connection.cursor(dictionary=True)
    try:
        cursor.execute(query)
        data = cursor.fetchall()
        print("Data Fetched Successfully")
        return data
    except Error as e:
        print(f"The error in SELECT query '{e}' occurred")
        return None
    
# Function to insert data into the database
def insert_func(connection, query, value):
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    try:
        # Execute the SQL query with the given value
        cursor.execute(query, value)
        # Commit the changes to the database
        connection.commit()
        # Return the number of rows inserted
        return cursor.rowcount
    except Error as e:
        # Print an error message if an exception occurs
        print(f"Error: {e}")
        # Return 0 if an error occurs
        return 0
    
connection= create_connection()

Connection to MySQL DB successful


In [175]:
#Create Database
create_db = "CREATE DATABASE Bank"


select_execute_query(connection, create_db)

The error in SELECT query '1007 (HY000): Can't create database 'Bank'; database exists' occurred


In [176]:
#Create Table customers

create_customers = """CREATE TABLE customers (
    Customer_id INT,
    transaction_amount INT,
    mobile_no VARCHAR(15),
    transaction_datetime DATETIME,
    Pincode INT(6)
);"""

select_execute_query(connection, create_customers)

The error in SELECT query '1050 (42S01): Table 'customers' already exists' occurred


In [154]:
create_state_master = """CREATE TABLE state_master (
officename varchar(100),pincode INT(6),officeType VARCHAR(100),
Deliverystatus VARCHAR(100),
divisionname VARCHAR(100),
regionname VARCHAR(100),
circlename VARCHAR(100),
Taluk VARCHAR(100),
Districtname VARCHAR(100),
statename VARCHAR(100)
)
;"""

select_execute_query(connection, create_state_master)

The error in SELECT query '1050 (42S01): Table 'state_master' already exists' occurred


### Generate the random data to import in customers table

In [155]:
import random
from datetime import datetime, timedelta
import pandas as pd

# Function to generate a random datetime object within a specified range
def generate_random_datetime(start_date, end_date):
    # Convert the start and end dates to timestamps
    start_timestamp = start_date.timestamp()
    end_timestamp = end_date.timestamp()
    # Generate a random timestamp within the range
    random_timestamp = random.uniform(start_timestamp, end_timestamp)
    # Convert the random timestamp back to a datetime object
    return datetime.fromtimestamp(random_timestamp)

# Define the start and end dates
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)

# Generate 100 random datetime objects between start_date and end_date
transaction_datetime = [generate_random_datetime(start_date, end_date) for _ in range(100)]
# Generate 100 random 10-digit integers for customer IDs
customer_id = [random.randrange(1000, 10000) for _ in range(100)]
# Generate 100 random 10-digit integers for mobile numbers
mobile_no = [random.randrange(10**9, 10**10) for _ in range(100)]
# Generate 100 random integers for transacting amounts
transaction_amount = [random.randrange(10000, 100000) for _ in range(100)]
# Generate 100 random 6-digit integers for pincodes
pincode = [random.randrange(100000, 1000000) for _ in range(100)]

# Create a dictionary to store the generated data
data = {
    "customer_id": customer_id,
    "transaction_amount": transaction_amount,
    "mobile_no": mobile_no,
    "transaction_datetime": transaction_datetime,
    "pincode": pincode
}

# Convert the data dictionary to a pandas DataFrame
df = pd.DataFrame(data)

### Insert the record in customers table

In [156]:
# Initialize a variable to keep track of the total number of records inserted
total_records_inserted = 0

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # SQL query to insert data into the customers table
    query = "INSERT INTO customers (customer_id, transaction_amount, mobile_no, transaction_datetime, pincode ) VALUES (%s, %s, %s, %s, %s)"
    # Values to be inserted into the table
    value = (row['customer_id'], row['transaction_amount'], row['mobile_no'], row['transaction_datetime'], row['pincode'])
    # Insert the data into the database and increment the total records inserted
    total_records_inserted += insert_func(connection, query, value)

# Print the total number of records inserted
print(f"Total records inserted: {total_records_inserted}")

# Close the database connection
if connection:
    connection.close()
    print("Database connection closed.")

Total records inserted: 100
Database connection closed.


### Import the record in the state_master table

In [185]:
pincode_data = 'https://data.gov.in/sites/default/files/all_india_pin_code.csv'
df1 = pd.read_csv("all_india_pin_code.csv", encoding='ISO-8859-1')
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154823 entries, 0 to 154822
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   officename      154823 non-null  object
 1   pincode         154823 non-null  int64 
 2   officeType      154823 non-null  object
 3   Deliverystatus  154823 non-null  object
 4   divisionname    154823 non-null  object
 5   regionname      154823 non-null  object
 6   circlename      154823 non-null  object
 7   Taluk           146490 non-null  object
 8   Districtname    154823 non-null  object
 9   statename       154823 non-null  object
dtypes: int64(1), object(9)
memory usage: 11.8+ MB
None


### Insert the pincode data into state_master table

In [189]:
# Initialize a variable to keep track of the total number of records inserted
total_records_inserted = 0

# Iterate over each row in the DataFrame
for index, row in df1.iterrows():
    # SQL query to insert data into the customers table
    query = "INSERT INTO state_master (officename, pincode, officeType, Deliverystatus, divisionname, regionname, circlename, Taluk, Districtname, statename) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    # Values to be inserted into the table
    value = (row['officename'], row['pincode'], row['officeType'], row['Deliverystatus'], row['divisionname'], row['regionname'], row['circlename'], row['Taluk'], row['Districtname'], row['statename'])
    # Insert the data into the database and increment the total records inserted
    total_records_inserted += insert_func(connection, query, value)

# Print the total number of records inserted
print(f"Total records inserted: {total_records_inserted}")

# Close the database connection
if connection:
    connection.close()
    print("Database connection closed.")

Total records inserted: 154823
Database connection closed.
